pytorch公式の畳み込みニューラルネットワークのexampleコードにコメントを追加したものです．

サンプルの畳み込みニューラルネットワークは，畳み込み層2つ，全結合層2つで構成されています．最初の畳み込み層conv1の出力はReLu関数を通し，次の畳み込み層conv2に入力されます．conv2の出力はReLU関数と通した後，Max poolingを行います．poolingと全結合層fc1の間は学習時dropout処理がされます．最初の全結合層fc1の出力はReLU関数を通し，次の全結合層fc2に入力されます．fc1とfc2間で学習時dropoutが起こります．fc2の出力はsoftmax関数により規格化されます．dropoutは学習時，任意の割合の入力ユニットの出力を0にします．それにより過学習を回避することが期待できます．しかし効果があるとは限りません．

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import time

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # まず最初の畳み込み層には1チャネルの入力が入る(nn.Conv2d(1, 32, 3, 1)のはじめの1)．画像認識の場合，入力のチャネルとは基本色の数と思っておく．
        # チャンネルが1つのときはグレースケール（輝度情報しかない）画像が入力で，チャネルが3つのときはRGBカラー画像が入力である．
        # 32は出力のチャネル数である(nn.Conv2d(1, 32, 3, 1)の32)．言い方を変えればカーネルの種類の数である．32種類のカーネルを適用した画像がこの畳込み層で出力されるといえる．詳しくは他のウェブサイトを参考にしてほしい．
        # 3は畳み込みのサイズ（カーネルサイズ）で，1はカーネルをズラす(stride)量である(nn.Conv2d(1, 32, 3, 1)の3と1)．詳しくは他のウェブサイトを参考にしてほしい．
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25) # dropout1では0.25の確率で出力の値が0になる．
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128) # 全結合層fc1は入力9216，出力128の中間層が作成される．
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x) # conv1の出力はReLU関数と通しconv2に入力される．
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2) # ReLU関数を通したconv2の出力はmax poolongされる．2はカーネルのサイズで，カーネルのサイズがstrideの数になる．
        x = self.dropout1(x)
        x = torch.flatten(x, 1) # 出力が1次元化される．
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        # fc2の出力はsoftmax関数にかけられる．しかし，誤差にcross entropyを用いる場合，誤差計算でlogの計算をするのではなく出力の段階でlogを計算する．
        output = F.log_softmax(x, dim=1)
        return output

In [3]:

def train(model, device, train_loader, optimizer, epoch):
    model.train()#学習モード
    # ミニバッチで学習がされる．1度の学習ですべてのデータを使うのではなく一部のデータを使う．
    #データローダが自動でデータをミニバッチに分けてくれる．
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)#データをデバイスに渡す．
        # 面白いことにtargetはラベル（例えば，1，2，...）なのだが，pytorchは自動でone-of-k codingにしてくれる．
        optimizer.zero_grad()#勾配の初期化
        output = model(data)#データをニューラルネットワークに入力し，出力を得る．
        loss = F.nll_loss(output, target) # 誤差を計算している．誤差関数はcross entropyだが，先にlogの計算を行っているのでnll_lossが用いられている．

        #学習する．
        loss.backward()#逆伝播
        optimizer.step()#パラメタ更新

        #ミニバッチごとに結果を表示する．
        #if batch_idx % 5 == 0:
        #    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #        epoch, batch_idx * len(data), len(train_loader.dataset),
        #        100. * batch_idx / len(train_loader), loss.item()))


In [4]:
def test(model, device, test_loader):
    model.eval()#推論モード
    test_loss = 0
    correct = 0
    with torch.no_grad():#torch.no_grad()は推論時に使う．余計な途中計算を保存しない．
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)#データをデバイスに渡す．
            output = model(data)#データをニューラルネットワークに入力し，出力を得る．
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # 誤差を足す．
            pred = output.argmax(dim=1, keepdim=True)  # 最大の出力を出すニューロンの番号を得る．数字画像の認識の場合，ニューロン番号が推定した数字になる．
                                                       # logは単調増加関数なので，softmaxの最大値を取るユニットはlog softmaxの最大値を取るユニットと等しい．
            correct += pred.eq(target.view_as(pred)).sum().item()#正解数を保存する．

    test_loss /= len(test_loader.dataset)#正解率を計算する．

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [5]:

# batch sizeの設定
train_kwargs = {'batch_size': 32}
test_kwargs = {'batch_size': 32}

# 入力画像の編集を行う関数．この例では規格化しかしていないが，回転などの処理も追加できる．
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

# データの読み込み
dataset1 = datasets.MNIST('.', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('.', train=False,
                       transform=transform)

# データローダにデータを渡す．データローダを使ってデータをミニバッチにする．
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [8]:
device = "cuda:0" # 計算するデバイスを指定している．gpuの方が計算が早く終る．
model = Net().to(device)# ニューラルネットワークモデルを読み込んで使用するデバイスに渡す．
optimizer = optim.Adadelta(model.parameters(), lr=0.5) # この例では最適化法としてAdadeltaが使われている．

Epochs = 10 # いわゆる，学習回数．
scheduler = StepLR(optimizer, step_size=1, gamma=0.7) # StepLRを用いることで学習回数が増えると学習率が減るようになる．


In [9]:
for epoch in range(1, Epochs + 1):
    start = time.time()#時間を取得する．
    train(model, device, train_loader, optimizer, epoch)#学習の関数を実行する．
    test(model, device, test_loader)#テストの関数を実行する．
    scheduler.step()#スケジューラーを呼ぶ．
    elapsed_time = time.time() - start#実行時間の計算をする．
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")#実行時間を表示する．
    

#学習済みモデルを保存する．
#model_path = 'checkpoint.pth'
#torch.save(model.to('cpu').state_dict(), model_path)

#Google Colabで学習したとき，学習済みモデルをダウンロードする．
#from google.colab import files
#files.download('checkpoint.pth')


Test set: Average loss: 0.0451, Accuracy: 9843/10000 (98%)

elapsed_time:18.15149688720703[sec]

Test set: Average loss: 0.0352, Accuracy: 9874/10000 (99%)

elapsed_time:16.77873182296753[sec]

Test set: Average loss: 0.0306, Accuracy: 9895/10000 (99%)

elapsed_time:16.80886435508728[sec]

Test set: Average loss: 0.0314, Accuracy: 9891/10000 (99%)

elapsed_time:16.863697052001953[sec]

Test set: Average loss: 0.0288, Accuracy: 9907/10000 (99%)

elapsed_time:16.827869415283203[sec]

Test set: Average loss: 0.0301, Accuracy: 9903/10000 (99%)

elapsed_time:16.76473116874695[sec]

Test set: Average loss: 0.0272, Accuracy: 9910/10000 (99%)

elapsed_time:16.919644832611084[sec]

Test set: Average loss: 0.0267, Accuracy: 9917/10000 (99%)

elapsed_time:16.907424449920654[sec]

Test set: Average loss: 0.0265, Accuracy: 9910/10000 (99%)

elapsed_time:16.70028281211853[sec]

Test set: Average loss: 0.0266, Accuracy: 9915/10000 (99%)

elapsed_time:16.68463969230652[sec]
